# SANS2D: I(Q) workflow for a single run (sample)

This notebook describes in detail the steps that are undertaken in the `sans.to_I_of_Q` workflow.

It assumes the detector data has been recorded in event mode, while the monitor data has been histogrammed.

The data used in this notebook has been published in [Manasi et al. (2021)](#manasi2021),
and we kindly thank the authors for allowing us to use their data.

**Note:** It uses sample run for simplicity and it is not intended to describe complete data reduction pipeline.
The complete pipeline is described in [SANS2D reduction](sans2d_reduction.ipynb).

**Outline:**

- We will begin by loading the data files containing the sample and the direct (empty sample holder) measurements.
- We will then apply some corrections to beamline components specific to the SANS2D beamline.
- This will be followed by some masking of some saturated or defect detector pixels
- Both sample and direct measurement, as well as their monitors, will then be converted to wavelength
- From the direct run, and the direct beam function, the normalization term will be computed
- Both sample measurement and normalization term will be converted to $Q$
- Finally, the sample counts (as a function of $Q$) will be divided by the normalization term (as a function of $Q$)

In [ ]:
import scipp as sc
import esssans as sans
from esssans.types import *
from esssans.logging import configure_workflow
import scippneutron as scn
import sciline

Set up the logging systems of scipp (including scippneutron and ess) and Mantid.

In [ ]:
logger = configure_workflow('sans2d_I_of_Q', filename='sans2d.log')

## Define reduction parameters

We define here whether to include the effects of gravity, and the binning in wavelength and in $Q$ to be used.

In [ ]:
from esssans.data import get_path

params = {}
params[NeXusMonitorName[Incident]] = 'monitor2'
params[NeXusMonitorName[Transmission]] = 'monitor4'
band = sc.linspace('wavelength', 2.0, 16.0, num=2, unit='angstrom')
params[WavelengthBands] = band
params[WavelengthBins] = sc.linspace(
    'wavelength', start=band[0], stop=band[-1], num=141
)
params[WavelengthMask] = sc.DataArray(
    data=sc.array(dims=['wavelength'], values=[True]),
    coords={
        'wavelength': sc.array(
            dims=['wavelength'], values=[2.21, 2.59], unit='angstrom'
        )
    },
)

params[QBins] = sc.linspace(dim='Q', start=0.01, stop=0.6, num=141, unit='1/angstrom')
params[Filename[BackgroundRun]] = get_path('SANS2D00063159.hdf5')
params[Filename[SampleRun]] = get_path('SANS2D00063114.hdf5')
params[Filename[DirectRun]] = get_path('SANS2D00063091.hdf5')
params[DirectBeamFilename] = get_path('DIRECT_SANS2D_REAR_34327_4m_8mm_16Feb16.hdf5')
params[BeamCenter] = sc.vector(value=[0.0945643, -0.082074, 0.0], unit='m')
params[NonBackgroundWavelengthRange] = sc.array(
    dims=['wavelength'], values=[0.7, 17.1], unit='angstrom'
)
params[CorrectForGravity] = True

In [ ]:
pipeline = sciline.Pipeline(sans.providers, params=params)

In [ ]:
scn.instrument_view(pipeline.compute(MaskedData[SampleRun]).hist(), pixel_size=0.0075)

In [ ]:
scn.instrument_view(
    pipeline.compute(Clean[SampleRun, Numerator]).value.hist(), pixel_size=0.0075
)

In [ ]:
scn.instrument_view(pipeline.compute(SolidAngle[SampleRun]), pixel_size=0.0075)

In [ ]:
denominator = pipeline.get(Clean[SampleRun, Denominator]).compute()
denominator.value.sum('spectrum').plot(norm='log')

In [ ]:
pipeline.get(BackgroundSubtractedIofQ).visualize(
    graph_attr={'size': '20,10', 'rankdir': 'LR'}
)

In [ ]:
pipeline.compute(IofQ[SampleRun]).hist().plot()
# sc.plot( sc.collapse(pipeline.compute(IofQ[SampleRun]).hist(), keep='Q'))

In [ ]:
pipeline.visualize(IofQ[SampleRun], graph_attr={'size': '12,10'})
pipeline.visualize(IofQ[BackgroundRun], graph_attr={'size': '12,10'})
pipeline.visualize(
    BackgroundSubtractedIofQ, graph_attr={'size': '20,10', 'rankdir': 'TD'}
)

In [ ]:
iofq = pipeline.compute(BackgroundSubtractedIofQ)

In [ ]:
%matplotlib widget
iofq.plot()

In [ ]:
pipeline.get(CleanMonitor[SampleRun, Incident]).visualize()

In [ ]:
pipeline.compute(CleanMonitor[SampleRun, Incident]).value

In [ ]:
pipeline.get(MaskedData[SampleRun]).visualize()

We now plot them on the same figure to asses the level of background noise

In [ ]:
sc.plot(monitors, norm='log', grid=True)

From this, we define a wavelength range between 0.7 &#8491; and 17.1 &#8491; where data is not considered to be background.

In [ ]:
scn.instrument_view(sample.hist(), pixel_size=0.0075)

## Beam center finder

The beam is not guaranteed to travel through the center of the detector panel,
and we thus have to apply a horizontal and vertical offset to our pixel positions so that the beam centre is at `x = y = 0`.
This is necessary for subsequent azimuthal averaging of the data counts into $Q$ bins.

The `beam_center` utility in the `sans` module is designed for this.
It requires us to define a $Q$ range over which convergence will be checked.

In [ ]:
q_range = sc.linspace('Q', 0.02, 0.3, 71, unit='1/angstrom')

center = sans.beam_center(
    data=sample,
    data_monitors=sample_monitors,
    direct_monitors=direct_monitors,
    wavelength_bins=wavelength_bins,
    q_bins=q_range,
    gravity=gravity,
)
print(center)

# Now shift pixels positions to get the correct beam center
sample.coords['position'] -= center